## 캡스톤 프로젝트 - 데이터 입력 작업 자동화 
- 여러 다른 데이터 입력 작업 형식을 옮기는 작업 
- 여기서 우리가 다룰 작업은 데이터 입력 작업 - 질로우 웹사이트에 부동산 가격 조사 
    - URL: https://www.zillow.com
- 개요 
    - 이번 캡스톤 프로젝트에서는 웹사이트와 웹스크래필에 대해 배운 내용을 모두 적용하여 bs4, Selenium으로 프로젝트 요구사항을 충족시켜 프로젝트를 완성하는 것이 목표이다.
- 과정 
    1. 구글 설문지에 새 입력양식 만들기 - https://docs.google.com/forms/에서 새 입력양식을 만들기
    2. 양식에 '단답형' 질문 3개 추가하기 
        - What's the address of the property?
        - What's the price per month?
        - What's the link to the property?
    3. '보내기'를 클릭하고, 양식의 링크 주소를 복사 하기. 프로그램을 만들 때 해당 주소 필요 
    4. 질로우(Zillow) 사이트의 이 페이지에서 웹사이트 구조를 확인하기. 여기 있는 데이터를 스크래핑하기
        - 프로그램 요구사항
            - BeautifulSoup과 requests를 사용하여 질로우 웹페이지에 있는 목록을 모두 스크래핑
            - 스크래핑한 항목들의 링크를 리스트로 만들기 
            - 스크래핑한 항목들의 가격을 리스트로 만들기
            - 스크래핑한 항목들의 주소를 리스트로 만들기 
            - 셀레니움으로 (위 1, 2, 3단계에서)생성한 입력양식을 채우기. 항목은 각각의 양식으로 저장하며, 양식에는 가격, 주소, 링크가 추가되어야 한다.
            - 데이터를 모두 입력하고 나면, 구글 설문지의 응답 탭에서 '구글 시트' 아이콘을 눌러 시트를 만들기. 부동산 정보가 담긴 스프레스 시트가 만들어진다.

In [ ]:
# 설문지 링크 
research_url = "https://docs.google.com/forms/d/e/1FAIpQLSdRAn7IbenPcd_LHRopwM0dY0J_y_3000Lcp9EiB9cZZ8XpCw/viewform?usp=sf_link"